# RG374 LegendPlex analysis IFNAR fl LysMcre CpG

In [1]:
options(warn=-1)

In [2]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
                         
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel), 
            
            # Statistics
            library(emmeans)

        )
    )
)

In [3]:
random_seed <- 42
set.seed(random_seed)

In [4]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [5]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter 

In [6]:
color$genotype <- c("D0 +/+"="#FFFFFF", "D0 cre/+"="#998989", "D3 +/+"="#FFFFFF", "D3 cre/+"="#998989", "D6 +/+"="#FFFFFF", "D6 cre/+"="#998989")

# Helper function

In [58]:
replace_greek_abg <- function(x) {
    
    map <- c(
        
        "\u03B1"="a", "\u0391"="A",
        "\u03B2"="b",  "\u0392"="B",
        "\u03B3"="g", "\u0393"="G"
    
    )
    
    f <- function(s) {
        
        s <- enc2utf8(s)
        for (k in names(map)) s <- gsub(k, map[[k]], s, fixed = TRUE)
        return(s)
    
    }
    
    if (is.factor(x)) {
        
        lev <- vapply(levels(x), f, "", USE.NAMES = FALSE)
        levels(x) <- lev
        return(x)
        
    } else if (is.list(x)) {
        
        lapply(x, replace_greek_abg)
        
    } else {
        
        vapply(x, f, "", USE.NAMES = FALSE)
    
    }

}

In [59]:
bar_plot <- function(mat, x, y, xlab="", ylab="", ggtitle="", scale_y=FALSE, parse_y=FALSE, expand_y=NULL, position_bar="stack", position_jitter="jitter", position="default") {

    ggtitle <- replace_greek_abg(ggtitle)
    
    if(scale_y) {
        
        res <- scale_label(mat, deparse(substitute(y)))
        mat <- res[[1]]
        ylab <- res[[2]]

    }

    if(position=="dodge") {

        position_bar <- position_dodge(width=0.8)
        position_jitter <- position_dodge(width=0.8)
    }
    
    p <- ggplot(mat, aes(x={{x}}, y={{y}}, color=genotype, fill=genotype)) + 
    
        geom_bar(stat="summary", width=0.8, color="black", size=0.1, fun=mean, position=position_bar) + 
        geom_jitter(shape=21, stroke=0.1, size=1.0, show.legend=FALSE, color="black", position=position_jitter) + 
        xlab(xlab) + ylab(ylab) + ggtitle(ggtitle) + 
        scale_color_manual(values=color$genotype) + 
        scale_fill_manual(values=color$genotype) + 
        theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
        guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))
    
    if(parse_y) {
        
        p <- p + ylab(parse(text=ylab))
        
    }


    if(!is.null(expand_y)) {

         p <- p + ylim(0, (max(ggplot_build(p)$data[[1]]$y) + expand_y))
    }
    
    return(p)
    
}

# Plasma 

In [60]:
mat <- read.csv("data/RG374_ifnar_fl_lysmcre_cpg/legendplex/plasma/RG374_ifnar_fl_lysmcre_cpg_plasma_legendplex_data_2024_08_23.csv") %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    rename_with(~gsub(".", "-", ., fixed=TRUE)) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cytokine", value.name=c("concentration")) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [61]:
bar_plot_1 <- lapply(split(mat, f=mat$cytokine), function(x) bar_plot(x, genotype, concentration, ylab="pg/ml", ggtitle=x$cytokine[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [62]:
pdf("result/figures/dev/bp_legendplex_plasma.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

# Spleen 

In [63]:
mat <- read.csv("data/RG374_ifnar_fl_lysmcre_cpg/legendplex/spleen/RG374_ifnar_fl_lysmcre_cpg_spleen_legendplex_data_2024_08_23.csv") %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    rename_with(~gsub(".", "-", ., fixed=TRUE)) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cytokine", value.name=c("concentration")) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [64]:
bar_plot_1 <- lapply(split(mat, f=mat$cytokine), function(x) bar_plot(x, genotype, concentration, ylab="pg/ml", ggtitle=x$cytokine[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [65]:
pdf("result/figures/dev/bp_legendplex_spleen.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2